# Build the Neural Network

**ネスト構造**  
小さなモジュールを組み合わせて大きなモジュールを作成する。  
ニューラルネットワーク自体がモジュール（部品、関数やクラスのように小分けにして再利用しやすいようにしている）であり、以下のような階層に分かれている。  
- レイヤー（最小単位）  
nn.Moduleの一種である、nn.Linear（全結合層）や nn.Conv2d（畳み込み層）など。
- ブロック（中間単位）  
複数のレイヤーをまとめたもの  

小さなモジュールを組み合わせて、大きなモジュールを作成する。  
　例）  
```nn.Linear```　　全結合層  
```nn.ReLU```　　　活性化関数  
```nn.Conv2d```　　畳み込み層  

In [62]:
from pathlib import Path
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## デバイスを作成

利用できるアクセラレータ（処理を行うデバイス）を設定

In [63]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else 'cpu'
print(f"Using {device} device")

Using cpu device


## クラスを定義

In [64]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()              # nn.Moduleの機能を呼び出す
        self.flatten = nn.Flatten()     # 二次元の画像を1次元の配列にならす。Linearは1次元しか受け取れないらしい
                                        # 入力が28*28ならば、784ピクセルになる
        self.linear_relu_stack = nn.Sequential(     # 複数のレイヤーを順番に実行するためのまとめた箱
            nn.Linear(28*28, 512),      # 入力784を512個の特徴量に変換
            nn.ReLU(),                  # 活性化関数
            nn.Linear(512, 512),        # 512を受け取って、512を返す
            nn.ReLU(),
            nn.Linear(512, 10),         # 全結合層として、10ラベルに分類し予測する
        )
    
    def forward(self, x):
        x = self.flatten(x)             # データを1列に変えて、
        logits = self.linear_relu_stack(x)          # レイヤーに通す。
        return logits                               # ロジット

- ロジット  
確率になる前の生の結果数値。  
例えば、[10.2, -2.1, 0.5, ...] のように出力され、一番大きい数値のインデックスが「モデルの予想（答え）」になる。

In [65]:
model = NeuralNetwork().to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [66]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)

pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)

print(f"Predicted class :{y_pred}")
print(logits.shape)
print(logits)

Predicted class :tensor([7])
torch.Size([1, 10])
tensor([[ 0.0088, -0.0232, -0.0939,  0.0500, -0.0589, -0.0120, -0.1291,  0.0833,
          0.0411, -0.0291]], grad_fn=<AddmmBackward0>)


予測ラベルのロジットを確認すると、確かに一番大きい値となっていることがわかる。

## 上記の中身を詳しく見ていこう

### モデルの各レイヤー

サンプルミニバッチを準備し、ネットワークに通してみる

In [67]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())
#input_image

torch.Size([3, 28, 28])


#### nn.Flatten  

28*28の2d画像を784個の1次元配列に変換

In [68]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())
#flat_image

torch.Size([3, 784])


In [69]:
len(flat_image[0])

784

上記をみてわかる通り、赤青緑それぞれを表す列があり、それらは784個の要素をもとに構成された。

#### nn.Linear

保存された重みとバイアスを使用して入力を線形変換。  

$$y = xA^T + b$$

x: 入力したデータ  
A: 重み（重要視するポイント（要素））  
b: バイアス  
y: 出力  

入力データをもとに、重要な特徴部分を残して（まとめて？）20個の出力としている。


$$\text{Output (20)} = \text{Input (784)} \times \text{Weights (784, 20)} + \text{Bias (20)}$$

In [70]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


layer1という入力（in_features）を合わせて、出力（out_features）を指定し、層を作成する。  
前処理後(flat_image)を作成した層にいれることで、内部で線形化の計算が行われ、結果がhidden1に収納される。

In [71]:
layer1.weight.shape

torch.Size([20, 784])

In [72]:
layer1.weight#[0][:5]

Parameter containing:
tensor([[-0.0124,  0.0262, -0.0194,  ...,  0.0211, -0.0175, -0.0225],
        [ 0.0006, -0.0347,  0.0132,  ..., -0.0345,  0.0082, -0.0051],
        [ 0.0221,  0.0275,  0.0219,  ...,  0.0032,  0.0087,  0.0160],
        ...,
        [ 0.0267, -0.0125, -0.0141,  ..., -0.0022,  0.0210,  0.0241],
        [-0.0054, -0.0048, -0.0256,  ...,  0.0297, -0.0258,  0.0161],
        [ 0.0046, -0.0270,  0.0301,  ..., -0.0252, -0.0292,  0.0233]],
       requires_grad=True)

In [73]:
len(layer1.weight), len(layer1.weight[0])

(20, 784)

20個中1個の出力を得るための784要素分の重みが含まれているという事だろうか。

#### nn.ReLU

非線形活性化関数

In [74]:
print(f"Before ReLU: {hidden1}\n")

hidden1 = nn.ReLU()(hidden1)

print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2835, -0.0899,  0.1448, -0.1582,  0.4209,  0.3880,  0.0963,  0.7778,
          0.2744,  0.0863,  0.0157,  0.5735, -0.1913,  0.2347, -0.0427, -0.3022,
         -0.1769, -0.0824, -0.1296, -0.0221],
        [ 0.3179, -0.0629, -0.0704, -0.1931,  0.2399,  0.1597,  0.0652,  0.7289,
         -0.0731,  0.2175,  0.0706,  0.4341, -0.3429,  0.0416,  0.1350, -0.6069,
         -0.0045,  0.1441,  0.1868, -0.1669],
        [ 0.5328,  0.1028, -0.0874, -0.4444,  0.2351,  0.1717,  0.2036,  0.7154,
          0.2903,  0.2593,  0.1277,  0.2207, -0.0812, -0.0365, -0.0575, -0.4267,
         -0.0049, -0.2597,  0.1094, -0.3920]], grad_fn=<AddmmBackward0>)

After ReLU: tensor([[0.2835, 0.0000, 0.1448, 0.0000, 0.4209, 0.3880, 0.0963, 0.7778, 0.2744,
         0.0863, 0.0157, 0.5735, 0.0000, 0.2347, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.3179, 0.0000, 0.0000, 0.0000, 0.2399, 0.1597, 0.0652, 0.7289, 0.0000,
         0.2175, 0.0706, 0.4341, 0.0000, 0.0416, 0.135

#### nn.Sequential

上記のレイヤーをコンテナとしてまとめる

In [75]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(2, 28, 28)
logits = seq_modules(input_image)

#### nn.Softmax

ロジットとして、各クラスの予測確立を表す。  

**活性化関数**
- ReLU:  
中間層で使用される。0以下の数値は0に、それよりも大きい数値はそのまま返す。重要な情報のみを残し、あとは捨てる。
- Softmax:  
確率に変換。値の合計が1になる。
$$\sigma(z)_i = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}$$

In [76]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [87]:
pred_probab

tensor([[0.0882, 0.0987, 0.1148, 0.1328, 0.0846, 0.1045, 0.0869, 0.0925, 0.0849,
         0.1121],
        [0.0820, 0.1082, 0.1031, 0.1398, 0.0828, 0.1000, 0.0885, 0.0838, 0.0888,
         0.1230]], grad_fn=<SoftmaxBackward0>)

In [88]:
len(pred_probab)

2

?

In [89]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0247, -0.0233,  0.0085,  ...,  0.0079,  0.0107,  0.0352],
        [-0.0340,  0.0017, -0.0352,  ...,  0.0195, -0.0280,  0.0081]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0163,  0.0023], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0101, -0.0400, -0.0046,  ..., -0.0194,  0.0198,  0.0331],
        [ 0.0201,  0.0381,  0.0016,  ...,  0.0356,  0.0048,  0.0419]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 